# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse',engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df['message']
Y = df.drop(['id','message','original','genre'],axis=1)

In [4]:
# Check if there is a class whose values are all the same (0 or 1)
print(Y.shape)
Y.sum(axis=0)

(26215, 36)


related                   20093
request                    4474
offer                       118
aid_related               10860
medical_help               2084
medical_products           1313
search_and_rescue           724
security                    471
military                    860
child_alone                   0
water                      1672
food                       2923
shelter                    2314
clothing                    405
money                       604
missing_people              298
refugees                    875
death                      1194
other_aid                  3446
infrastructure_related     1705
transport                  1201
buildings                  1333
electricity                 532
tools                       159
hospitals                   283
shops                       120
aid_centers                 309
other_infrastructure       1151
weather_related            7297
floods                     2155
storm                      2443
fire    

In [5]:
# 有一列都是0会导致报错（一个class里需要至少两种值）
# In one class there should be at least 2 kinds of values
Y = Y.fillna(0).drop('child_alone',axis=1)

In [6]:
# get columns names
category_names = Y.columns.values
category_names

array(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related',
       'transport', 'buildings', 'electricity', 'tools', 'hospitals',
       'shops', 'aid_centers', 'other_infrastructure', 'weather_related',
       'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'], dtype=object)

### 2. Write a tokenization function to process your text data

In [7]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\meiji\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\meiji\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\meiji\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
def tokenize(text):
    stop_words = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    
    #Regex to find urls
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    # Finds all urls from the provided text
    detected_urls = re.findall(url_regex, text)
    #Replaces all urls found with the "urlplaceholder"
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    # Normalize
    text = re.sub(r"[^a-zA-Z0-9]"," ", text.lower())
    # Tokenize
    tokens = word_tokenize(text)
    # Lemmatize
    clean_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    #new_text = " ".join(tokens)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer


def build_pipeline(classifier):
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf',TfidfTransformer()),
        ('clf',MultiOutputClassifier(classifier))
    ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
from sklearn.model_selection import train_test_split


X_train,X_test,y_train,y_test = train_test_split(X,Y)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

def grid_search(pipeline,params):
    cv = GridSearchCV(pipeline,
                      param_grid=params,
                      n_jobs=-1,cv=3)
    return cv

In [13]:
from sklearn.svm import SVC

clf = SVC(class_weight={1:10})
params = {
        'clf__estimator__kernel': ['linear'],
        'clf__estimator__C'     : [1e-3,1e-2,1e-1,1]
    }


pipeline = build_pipeline(clf)
svc = grid_search(pipeline,params)

In [14]:
svc.fit(X_train, y_train)
svc.best_params_

{'clf__estimator__C': 1, 'clf__estimator__kernel': 'linear'}

In [9]:
# import joblib
# svc = joblib.load('classifier.pkl')

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

#### use classification report

In [25]:
from sklearn.metrics import classification_report,accuracy_score,f1_score,roc_auc_score

def evaluate_model(model, X_test, Y_test, category_names,print_result=True):
    y_pred = model.predict(X_test)
    n_class = len(category_names)
    avg_acc = 0
    avg_auc, avg_f1 = 0, 0
    for i in range(n_class):
        avg_acc += accuracy_score(Y_test.iloc[:, i].values, y_pred[:,i])
        avg_auc += roc_auc_score(Y_test.iloc[:, i].values, y_pred[:,i])
        avg_f1  += f1_score(Y_test.iloc[:, i].values, y_pred[:,i])
        if print_result:
            print("Category:",category_names[i])
            print(classification_report(Y_test.iloc[:, i].values, y_pred[:,i]))
            print("\n")
            
    avg_acc /= n_class
    avg_auc /= n_class
    avg_f1  /= n_class
    print(f"Average of accuracy:{avg_acc}")
    print(f"Average of auc:{avg_auc}")
    print(f"Average of f1:{avg_f1}")
    return avg_acc,avg_auc,avg_f1

In [26]:
evaluate_model(svc,X_test,y_test,category_names)

Category: related
              precision    recall  f1-score   support

           0       0.76      0.15      0.26      1567
           1       0.79      0.98      0.88      4987

    accuracy                           0.79      6554
   macro avg       0.77      0.57      0.57      6554
weighted avg       0.78      0.79      0.73      6554



Category: request
              precision    recall  f1-score   support

           0       0.94      0.85      0.89      5435
           1       0.51      0.76      0.61      1119

    accuracy                           0.83      6554
   macro avg       0.73      0.80      0.75      6554
weighted avg       0.87      0.83      0.85      6554



Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6521
           1       0.14      0.09      0.11        33

    accuracy                           0.99      6554
   macro avg       0.57      0.54      0.55      6554
weighted avg      

Category: aid_centers
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      6480
           1       0.16      0.09      0.12        74

    accuracy                           0.98      6554
   macro avg       0.58      0.54      0.56      6554
weighted avg       0.98      0.98      0.98      6554



Category: other_infrastructure
              precision    recall  f1-score   support

           0       0.97      0.95      0.96      6284
           1       0.19      0.25      0.22       270

    accuracy                           0.93      6554
   macro avg       0.58      0.60      0.59      6554
weighted avg       0.94      0.93      0.93      6554



Category: weather_related
              precision    recall  f1-score   support

           0       0.93      0.84      0.88      4767
           1       0.66      0.83      0.73      1787

    accuracy                           0.83      6554
   macro avg       0.79      0.83      0.81  

(0.9297876978072279, 0.7041076967707024, 0.44839617017807737)

#### use hamming 

In [12]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    https://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [ ]:
y_pred = svc.predict(X_test)
hamming_score(y_test,y_pred)

In [ ]:
# use the sklearn in-build hamming loss function
from sklearn.metrics import hamming_loss

hamming_loss(y_test,y_pred)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
from sklearn.naive_bayes import ComplementNB

clf = ComplementNB()
params = {
    'clf__estimator__alpha':[1e-3,1e-2,1e-1,1,10]
}


pipeline = build_pipeline(clf)
NB = grid_search(pipeline,params)

In [27]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier()
params = {
    'clf__estimator__n_neighbors':[5,10]
}


pipeline = build_pipeline(clf)
KNN = grid_search(pipeline,params)

In [28]:
KNN.fit(X_train, y_train)
KNN.best_params_

C:\Users\meiji\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [nan nan]
  category=UserWarning


{'clf__estimator__n_neighbors': 5}

In [15]:
# import joblib
# NB = joblib.load('classifier_nb.pkl')

In [29]:
evaluate_model(KNN,X_test,y_test,category_names)

Category: related
              precision    recall  f1-score   support

           0       0.70      0.11      0.19      1567
           1       0.78      0.99      0.87      4987

    accuracy                           0.78      6554
   macro avg       0.74      0.55      0.53      6554
weighted avg       0.76      0.78      0.71      6554



Category: request
              precision    recall  f1-score   support

           0       0.84      1.00      0.91      5435
           1       0.80      0.09      0.16      1119

    accuracy                           0.84      6554
   macro avg       0.82      0.54      0.53      6554
weighted avg       0.83      0.84      0.78      6554



Category: offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg      

C:\Users\meiji\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\meiji\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\meiji\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C

 shelter
              precision    recall  f1-score   support

           0       0.91      1.00      0.95      5963
           1       0.76      0.04      0.08       591

    accuracy                           0.91      6554
   macro avg       0.84      0.52      0.52      6554
weighted avg       0.90      0.91      0.88      6554



Category: clothing
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6457
           1       0.50      0.01      0.02        97

    accuracy                           0.99      6554
   macro avg       0.74      0.51      0.51      6554
weighted avg       0.98      0.99      0.98      6554



Category: money
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6418
           1       1.00      0.01      0.03       136

    accuracy                           0.98      6554
   macro avg       0.99      0.51      0.51      6554
weighted avg       0.98   

C:\Users\meiji\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\meiji\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\meiji\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6487
           1       0.00      0.00      0.00        67

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554



Category: shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6532
           1       0.00      0.00      0.00        22

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554



Category: aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6480
           1       0.00      0.00      0.00        74

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.9

C:\Users\meiji\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\meiji\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\meiji\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.9279698330354421, 0.510025983881546, 0.05856178122817416)

### 9. Export your model as a pickle file

In [31]:
import joblib

joblib.dump(svc,'classifier_svc_weighted.pkl')
joblib.dump(KNN,'classifier_knn.pkl')

['classifier_svc_weighted.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [23]:
import joblib
import numpy,pandas,sklearn 

cv = joblib.load('classifier_svc.pkl')
mes = "heavy rains! plz come and help with my neighbor and  there is a kid alone!"
cv.predict([mes])

array([[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]], dtype=int64)